## Importing libraries



In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.models import save_model
from tensorflow.keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

## Initialize the training and validation generators

In [6]:
# Assining the directory of our data 
train_dir = 'data/train'     
val_dir = 'data/test'

# Generating greyscale image data in terms of 0-1 of pixels 
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Reading our generated data from ImageDataGenerator
# flow_from_directory when we have sub folders in directory 
train_generator = train_datagen.flow_from_directory(
        # directory location
        train_dir,
        # Size of images to be selected 
        target_size=(48,48),
        # Number of images to be selected 
        batch_size=64,
        color_mode="grayscale",
        # Type of class to predict 
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


## Building the convolution network architecture

In [7]:
"""We have two options to create neural network in keras sequential and functional 
where sequential allows us to create model layer by layer in a sequence with only single 
input and single output.
"""
emotion_model = Sequential()

# Conv2D creates a convolution kernal which acts as a filter to produce tensor of outputs.
# Starting with 32 filters, matrix size of 3x3,
# Using relu(Rectified linear unit) as activation function which decides if neuron should activate or not.
# MaxPooling helps in downsampling our data.
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
# Droupout is used to prevent overfiting 
emotion_model.add(Dropout(0.25))

# Flatten is used to convert 2d array into 1d array
emotion_model.add(Flatten())
# Dense layer is simple layer of neurons which is used to 
# receives input from all the neurons of previous layer
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
# Softmax function is used for classification
emotion_model.add(Dense(7, activation='softmax'))

## Compiling and Training the model

In [8]:
# Loss function is used to find error or deviation in the learning process.
# Optimizer assigns the input weights by comparing the prediction and the loss function.
# Metrics is used to evaluate the performance of our model.
# lr is learning rate and decay is learning rate decay over each update.
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

# Now we train using .fit_generator() method.
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        # Total number of training examples present in a single batch.
        steps_per_epoch=28709 // 64,
        # Epoch is when an ENTIRE dataset is passed forward and backward 
        # through the neural network only ONCE.
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

c:\3-Programming\EmoDec\.emodec_venv\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\Asus\AppData\Local\Temp/ipykernel_5412/2449506319.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 53s 116ms/step - loss: 1.8280 - accuracy: 0.2476 - val_loss: 1.8294 - val_accuracy: 0.2478
Epoch 2/50
448/448 [==============================] - 42s 94ms/step - loss: 1.7976 - accuracy: 0.2526 - val_loss: 1.7808 - val_accuracy: 0.2693
Epoch 3/50
448/448 [==============================] - 40s 90ms/step - loss: 1.7268 - accuracy: 0.3014 - val_loss: 1.6224 - val_accuracy: 0.3763
Epoch 4/50
448/448 [==============================] - 42s 93ms/step - loss: 1.6048 - accuracy: 0.3737 - val_loss: 1.5344 - val_accuracy: 0.4121
Epoch 5/50
448/448 [==============================] - 40s 88ms/step - loss: 1.5375 - accuracy: 0.4021 - val_loss: 1.4901 - val_accuracy: 0.4174
Epoch 6/50
448/448 [==============================] - 40s 90ms/step - loss: 1.4881 - accuracy: 0.4242 - val_loss: 1.4352 - val_accuracy: 0.4528
Epoch 7/50
448/448 [==============================] - 42s 94ms/step - loss: 1.4446 - accuracy: 0.4437 - val_loss: 1.4016 - val_accuracy

## Saving the model weights

In [9]:
save_model(emotion_model,'emotion_model.h5')